# Aula Teórica 3 (guião)
### Semana de 6 a 10 de Março de 2023
### José Carlos Ramalho
### Sinopsis:

* Expressões regulares: o módulo re em Python;
* Construção de algumas aplicações;
* Introdução aos analisadores léxicos.

## Expressões Regulares em Python

### Simulação do scanf()

In [ ]:
# Ler um inteiro: scanf("%d", &num)

import re

inteiro = re.compile(r'^(\+|-)?(\d+)$')
valor = input("Introduza um inteiro: ")
if inteiro.fullmatch(valor):
    print("Li um inteiro válido: ", valor)
else:
    print("Inteiro inválido: ", valor)

In [ ]:
# Desafio ler um real tipo: scanf("%.2f")

import re

real = re.compile(r'^(\+|-)?(\d+)(\.\d\d)?$')
valor = input("Introduza um número real com até duas casas decimais: ")
if real.fullmatch(valor):
    print("Li um real válido: ", valor)
else:
    print("Real inválido: ", valor)

In [4]:
# Desafio: ler uma lista de elementos (sequência de alfanuméricos) 
# separados por ',' e podendo ter espaços pelo meio: batatas, 12, cebolas, 23, abacate

import re

lista = re.compile(r'^\s*\w+(\s*,\s*\w+)*$')
valor = input("Introduza uma lista de elementos separados por ,: ")
if lista.fullmatch(valor):
    print("Li uma lista válida: ", valor)
else:
    print("Lista inválida: ", valor)

Li uma lista válida:  12,78,987,5


In [5]:
# Separar os elementos da lista e colocá-los numa lista do Python
import re
mylista = re.split(r'\s*,\s*', valor)
print(mylista)

['12', '78', '987', '5']


### TPC2: Somador on/off

![AFD do somador](somador2.png)

In [6]:
# Sem o re

texto = ''' 
Este é apenas 1 texto demonstrativo.
Amanhã, 2023-03-07, tens de ir visitar o teu amigo.
Até agora totalizamos: =
ofF
Agora já não está a somar os números: 1, 2, ...
On, ligou-se outra vez, cuidado com o 1, 2, 3 ...
Chegamos ao fim com =
'''

on = True
total = 0
ptr = 0

while ptr < len(texto):
    if texto[ptr] == '=':
        print('Total = ', total)
        ptr = ptr + 1
    elif texto[ptr] in "oO":
        ptr = ptr + 1
        if texto[ptr] in "nN":
            on = True
            ptr = ptr + 1
        elif texto[ptr] in "fF":
            ptr = ptr + 1
            if texto[ptr] in "fF":
                on = False
                ptr = ptr + 1
    elif texto[ptr] in "0123456789":
        valor = int(texto[ptr])
        ptr = ptr + 1
        while texto[ptr] in "0123456789":
            valor = valor*10 + int(texto[ptr])
            ptr = ptr + 1
        if on:
            total = total + valor
    else:
        ptr = ptr + 1



Total =  2034
Total =  2040


In [45]:
# Com o re
import re

texto = ''' 
Este é apenas 1 texto demonstrativo.
Amanhã, 2023-03-07, tens de ir visitar o teu amigo.
Até agora totalizamos: =
ofF
Agora já não está a somar os números: 1, 2, ...
On, ligou-se outra vez, cuidado com o 1, 2, 3 ...
Chegamos ao fim com =
'''

token_specification = [
        ('INT', r'(\+|-)?\d+'),             
        ('ON',  r'(?i:on)'),             
        ('OFF', r'(?i:off)'),
        ('EQ', r'='),
        ('SKIP', r'\s+'),
        ('UNKNOWN', r'.')
    ]

mypattern = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
print(mypattern)
m = re.match(mypattern, texto)
print(m) 
print(m.start())
print(m.end())
print(m.groups())
print(m.groupdict())

(?P<INT>(\+|-)?\d+)|(?P<ON>(i:on))|(?P<OFF>(i:off))|(?P<EQ>=)|(?P<SKIP>\s+)|(?P<UNKOWN>.)
<re.Match object; span=(0, 2), match=' \n'>
0
2
(None, None, None, None, None, None, None, ' \n', None)
{'INT': None, 'ON': None, 'OFF': None, 'EQ': None, 'SKIP': ' \n', 'UNKOWN': None}


In [66]:
# Com o re
import re

texto = ''' 
Este é apenas 1 texto demonstrativo.
Amanhã, 2023-03-07, tens de ir visitar o teu amigo.
Até agora totalizamos: =
ofF
Agora já não está a somar os números: 1, 2, ...
Ora vamos ver: =
On, ligou-se outra vez, cuidado com o 1, 2, 3 ...
Chegamos ao fim com =
'''

token_specification = [
        ('INT', r'(\+|-)?\d+'),             
        ('ON',  r'(?i:on)'),             
        ('OFF', r'(?i:off)'),
        ('EQ', r'='),
        ('SKIP', r'\s+'),
        ('UNKNOWN', r'.')
    ]

mypattern = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)

on = True
total = 0
ptr = 0

while ptr < len(texto):
    m = re.match(mypattern, texto[ptr:])
    token = m.groupdict()
    ptr = ptr + m.end()
    if token['EQ']:
        print('Total = ', total)
    elif token['ON']:
        on = True
    elif token['OFF']:
        on = False
    elif token['INT']:
        if on:
            valor = int(m.group(1))
            total = total + valor
    elif token['SKIP']:
        pass
    elif token['UNKNOWN']:
        pass



Total =  2014
Total =  2014
Total =  2020


In [71]:
# Com o re.findall()

import re

texto = ''' Este é apenas 1 texto demonstrativo. Amanhã, 2023-03-07, tens de ir visitar o teu amigo.
Até agora totalizamos: = ofF  Agora já não está a somar os números: 1, 2, ...
Ora vamos ver: = On, ligou-se outra vez, cuidado com o 1, 2, 3 ...  Chegamos ao fim com = '''

ON = True
total = 0

for i,on,off,equ,skip,unk in re.findall(r'''
         ([+-]?\d+)             
      |  (on)             
      |  (off)
      |  (=)
      |  (\s+)   #...podíamos omitir
      |  (.)     #...podíamos omitir
    ''', texto, re.I|re.X):
 
    if   equ      : print('Total = ', total)
    elif on       : ON = True
    elif off      : ON = False
    elif i and ON : total = total + int(i)
    #  elif skip  : pass
    #  elif unk   : pass

Total =  2014
Total =  2014
Total =  2020


In [6]:
# Com o re.finditer() e numa abordagem diferente
import re

texto = ''' Este é apenas 1 texto demonstrativo. Amanhã, 2023-03-07, tens de ir visitar o teu amigo.
Até agora totalizamos: = ofF  Agora já não está a somar os números: 1, 2, ...
Ora vamos ver: = On, ligou-se outra vez, cuidado com o 1, 2, 3 ...  Chegamos ao fim com = '''

inteiro = r'(\+|-)?(\d+)'
pint = re.compile(inteiro)
sinal = 1
total = 0

encontrados = pint.finditer(texto)
for m in encontrados:
    print(m, m.groups())

<re.Match object; span=(15, 16), match='1'> (None, '1')
<re.Match object; span=(46, 50), match='2023'> (None, '2023')
<re.Match object; span=(50, 53), match='-03'> ('-', '03')
<re.Match object; span=(53, 56), match='-07'> ('-', '07')
<re.Match object; span=(158, 159), match='1'> (None, '1')
<re.Match object; span=(161, 162), match='2'> (None, '2')
<re.Match object; span=(223, 224), match='1'> (None, '1')
<re.Match object; span=(226, 227), match='2'> (None, '2')
<re.Match object; span=(229, 230), match='3'> (None, '3')


In [14]:
# Com o re.finditer() 

texto = ''' Este é apenas 1 texto demonstrativo. Amanhã, 2023-03-07, tens de ir visitar o teu amigo.
Até agora totalizamos: = ofF  Agora já não está a somar os números: 1, 2, ...
Ora vamos ver: = On, ligou-se outra vez, cuidado com o 1, 2, 3 ...  Chegamos ao fim com = '''

ON = True
total = 0
tokens = re.finditer(r'''
         ([+-]?\d+)             
      |  (on)             
      |  (off)
      |  (=)
      |  (\s+)   #...podíamos omitir
      |  (.)     #...podíamos omitir
    ''', texto, re.I|re.X)

for t in tokens:
    i,on,off,equ,skip,unk = t.groups()
 
    if   equ      : print('Total = ', total)
    elif on       : ON = True
    elif off      : ON = False
    elif i and ON : total = total + int(i)
    #  elif skip  : pass
    #  elif unk   : pass

Total =  2014
Total =  2014
Total =  2020


## Análise Léxica

Símbolos terminais:
1. Sinais: são constituídos por um caráter;
2. Palavras reservadas: strings constantes;
3. Terminais variáveis: identificadores, inteiros, etc.

``` 
a = 34 + 23-7*2

ID ATRIB INT OP INT OP INT OP INT

id = r'[_A-Za-z]\w*'
iss = r'\d+'
eq = r'='
op = r'[+\-*/]'



AFND 
AFD = (S, Z, Estados, funçãoTransição)

    d   _   letra   
--------------------
S   B   A   A
B   B   -   -
A   A   A   A

inicial = S
final = [A,B] 
estado = inicial

for t in input:
    estado = tabela[estado, t]
if estado in final:
    Sucesso
```

## Ply: `https://github.com/dabeaz/ply/blob/master/doc/ply.md`

* lex.py
* yacc.py

### Lex

`In: x = 3 + 42 * (s - t)`

`Out: 'x','=', '3', '+', '42', '*', '(', 's', '-', 't', ')'`

#### Nomeamos os tokens

`'ID','EQUALS','NUMBER','PLUS','NUMBER','TIMES', 'LPAREN','ID','MINUS','ID','RPAREN'`

```
('ID','x'), ('EQUALS','='), ('NUMBER','3'), 
('PLUS','+'), ('NUMBER','42'), ('TIMES','*'),
('LPAREN','('), ('ID','s'), ('MINUS','-'),
('ID','t'), ('RPAREN',')')
```

In [ ]:
# Exemplo do manual
# ------------------------------------------------------------
# calclex.py
#
# tokenizer for a simple expression evaluator for
# numbers and +,-,*,/
# ------------------------------------------------------------
import ply.lex as lex

# List of token names.   This is always required
tokens = (
   'NUMBER',
   'PLUS',
   'MINUS',
   'TIMES',
   'DIVIDE',
   'LPAREN',
   'RPAREN',
)

# Regular expression rules for simple tokens
t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_LPAREN  = r'\('
t_RPAREN  = r'\)'

# A regular expression rule with some action code
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)    
    return t

# Define a rule so we can track line numbers
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

# A string containing ignored characters (spaces and tabs)
t_ignore  = ' \t'

# Error handling rule
def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)

# Build the lexer
lexer = lex.lex()

# Test it out
data = '''
3 + 4 * 10
  + -20 *2
'''

# Give the lexer some input
lexer.input(data)

for tok in lexer:
    print(tok)
    print(tok.type, tok.value, tok.lineno, tok.lexpos)